In [ ]:
import math

In [ ]:
def get_vibrant_contrasting_color(background_rgb, saturation=0.8, lightness_range=(0.3, 0.7)):
    """Generate a vibrant contrasting color using complementary hues with controlled luminance."""
    r, g, b = [x/255.0 for x in background_rgb]
    
    # Convert RGB to HSL
    c_max = max(r, g, b)
    c_min = min(r, g, b)
    delta = c_max - c_min
    
    # Calculate hue (0-360)
    if delta == 0:
        h = 0
    elif c_max == r:
        h = 60 * (((g - b) / delta) % 6)
    elif c_max == g:
        h = 60 * (((b - r) / delta) + 2)
    else:
        h = 60 * (((r - g) / delta) + 4)
    
    # Use complementary hue (opposite side of color wheel)
    comp_hue = (h + 180) % 360
    
    # Calculate background luminance to determine optimal foreground lightness
    L_background = 0.2126 * r + 0.7152 * g + 0.0722 * b
    
    # Choose lightness based on background luminance
    if L_background > 0.6:
        target_lightness = lightness_range[0]  # Darker color for light backgrounds
    else:
        target_lightness = lightness_range[1]  # Lighter color for dark backgrounds
    
    # Convert HSL back to RGB
    return hsl_to_rgb(comp_hue, saturation, target_lightness)

def hsl_to_rgb(h, s, l):
    """Convert HSL color to RGB"""
    h = h / 360.0
    if s == 0:
        r = g = b = l
    else:
        def hue_to_rgb(p, q, t):
            t += 1 if t < 0 else 0
            t -= 1 if t > 1 else 0
            if t < 1/6: return p + (q - p) * 6 * t
            if t < 1/2: return q
            if t < 2/3: return p + (q - p) * (2/3 - t) * 6
            return p
        
        q = l * (1 + s) if l < 0.5 else l + s - l * s
        p = 2 * l - q
        
        r = hue_to_rgb(p, q, h + 1/3)
        g = hue_to_rgb(p, q, h)
        b = hue_to_rgb(p, q, h - 1/3)
    
    return tuple(int(x * 255) for x in (r, g, b))

def get_cielab_contrasting_color(background_rgb, target_lightness_diff=30):
    """Generate contrasting color using CIELAB color space for better perceptual contrast."""
    
    def rgb_to_xyz(rgb):
        r, g, b = [x/255.0 for x in rgb]
        # Convert to linear RGB
        r = r/12.92 if r <= 0.04045 else ((r+0.055)/1.055)**2.4
        g = g/12.92 if g <= 0.04045 else ((g+0.055)/1.055)**2.4
        b = b/12.92 if b <= 0.04045 else ((b+0.055)/1.055)**2.4
        # Convert to XYZ
        x = r*0.4124 + g*0.3576 + b*0.1805
        y = r*0.2126 + g*0.7152 + b*0.0722
        z = r*0.0193 + g*0.1192 + b*0.9505
        return (x, y, z)
    
    def xyz_to_lab(xyz):
        x, y, z = xyz
        # D65 reference white
        xn, yn, zn = 0.95047, 1.00000, 1.08883
        # Convert to LAB
        x, y, z = x/xn, y/yn, z/zn
        x = x**(1/3) if x > 0.008856 else 7.787*x + 16/116
        y = y**(1/3) if y > 0.008856 else 7.787*y + 16/116
        z = z**(1/3) if z > 0.008856 else 7.787*z + 16/116
        L = 116*y - 16
        a = 500*(x - y)
        b = 200*(y - z)
        return (L, a, b)
    
    def lab_to_xyz(lab):
        L, a, b = lab
        # Convert back to XYZ
        y = (L + 16) / 116
        x = a / 500 + y
        z = y - b / 200
        # Convert to linear
        x = x**3 if x**3 > 0.008856 else (x - 16/116) / 7.787
        y = y**3 if y**3 > 0.008856 else (y - 16/116) / 7.787
        z = z**3 if z**3 > 0.008856 else (z - 16/116) / 7.787
        # D65 reference white
        xn, yn, zn = 0.95047, 1.00000, 1.08883
        return (x*xn, y*yn, z*zn)
    
    def xyz_to_rgb(xyz):
        x, y, z = xyz
        # Convert to linear RGB
        r = x*3.2406 + y*-1.5372 + z*-0.4986
        g = x*-0.9689 + y*1.8758 + z*0.0415
        b = x*0.0557 + y*-0.2040 + z*1.0570
        # Apply gamma correction
        r = 12.92*r if r <= 0.0031308 else 1.055*(r**(1/2.4)) - 0.055
        g = 12.92*g if g <= 0.0031308 else 1.055*(g**(1/2.4)) - 0.055
        b = 12.92*b if b <= 0.0031308 else 1.055*(b**(1/2.4)) - 0.055
        # Clamp and convert to 0-255
        r = max(0, min(1, r))
        g = max(0, min(1, g))
        b = max(0, min(1, b))
        return tuple(int(x * 255) for x in (r, g, b))
    
    # Convert background to LAB
    bg_lab = xyz_to_lab(rgb_to_xyz(background_rgb))
    L_bg, a_bg, b_bg = bg_lab
    
    # Generate contrasting color by inverting lightness and keeping similar chroma
    if L_bg > 50:
        L_new = max(20, L_bg - target_lightness_diff)  # Darker
    else:
        L_new = min(80, L_bg + target_lightness_diff)  # Lighter
    
    # Use opposite a and b values for more vibrant contrast
    a_new = -a_bg * 0.7
    b_new = -b_bg * 0.7
    
    # Convert back to RGB
    return xyz_to_rgb(lab_to_xyz((L_new, a_new, b_new)))

def get_adaptive_contrasting_color(background_rgb, hue_shift=150, min_contrast_ratio=4.5):
    """Generate contrasting color with controlled hue shift and minimum contrast ratio."""
    
    def calculate_contrast_ratio(rgb1, rgb2):
        # Calculate relative luminance
        def get_luminance(rgb):
            r, g, b = [x/255.0 for x in rgb]
            r = r/12.92 if r <= 0.03928 else ((r+0.055)/1.055)**2.4
            g = g/12.92 if g <= 0.03928 else ((g+0.055)/1.055)**2.4
            b = b/12.92 if b <= 0.03928 else ((b+0.055)/1.055)**2.4
            return 0.2126*r + 0.7152*g + 0.0722*b
        
        L1 = get_luminance(rgb1)
        L2 = get_luminance(rgb2)
        lighter = max(L1, L2)
        darker = min(L1, L2)
        return (lighter + 0.05) / (darker + 0.05)
    
    r, g, b = [x/255.0 for x in background_rgb]
    
    # Convert to HSL
    c_max = max(r, g, b)
    c_min = min(r, g, b)
    delta = c_max - c_min
    
    if delta == 0:
        h = 0
    elif c_max == r:
        h = 60 * (((g - b) / delta) % 6)
    elif c_max == g:
        h = 60 * (((b - r) / delta) + 2)
    else:
        h = 60 * (((r - g) / delta) + 4)
    
    l = (c_max + c_min) / 2
    s = 0 if delta == 0 else delta / (1 - abs(2*l - 1))
    
    # Apply hue shift
    new_hue = (h + hue_shift) % 360
    
    # Adjust saturation and lightness for optimal contrast
    if l > 0.7:  # Light background
        new_lightness = 0.3  # Dark foreground
        new_saturation = min(1.0, s + 0.3)  # Boost saturation
    else:  # Dark background
        new_lightness = 0.8  # Light foreground  
        new_saturation = min(1.0, s + 0.2)  # Moderate saturation boost
    
    # Convert back to RGB
    candidate_color = hsl_to_rgb(new_hue, new_saturation, new_lightness)
    
    # Verify contrast ratio
    contrast = calculate_contrast_ratio(background_rgb, candidate_color)
    
    # Adjust if contrast is insufficient
    if contrast < min_contrast_ratio:
        if l > 0.7:  # Light background
            new_lightness = max(0.1, new_lightness - 0.1)
        else:  # Dark background
            new_lightness = min(0.9, new_lightness + 0.1)
        candidate_color = hsl_to_rgb(new_hue, new_saturation, new_lightness)
    
    return candidate_color

def vibrant_contrasting_color(bg_rgb):
    def rgb_to_hsl(r, g, b):
        r, g, b = r/255, g/255, b/255
        cmax = max(r, g, b)
        cmin = min(r, g, b)
        delta = cmax - cmin
        
        # Hue calculation
        if delta == 0:
            h = 0
        elif cmax == r:
            h = 60 * (((g - b) / delta) % 6)
        elif cmax == g:
            h = 60 * (((b - r) / delta) + 2)
        else:
            h = 60 * (((r - g) / delta) + 4)
        
        # Lightness
        l = (cmax + cmin) / 2
        
        # Saturation
        s = 0 if delta == 0 else delta / (1 - abs(2*l - 1))
        
        return h % 360, s, l
    
    def relative_luminance(r, g, b):
        """Calculate relative luminance per WCAG 2.1 (sRGB)"""
        def linearize(channel):
            # Normalize to [0, 1] and linearize gamma
            c = channel / 255.0
            return c / 12.92 if c <= 0.03928 else ((c + 0.055) / 1.055) ** 2.4
        
        r_lin = linearize(r)
        g_lin = linearize(g)
        b_lin = linearize(b)
        
        # Apply luminance coefficients
        return 0.2126 * r_lin + 0.7152 * g_lin + 0.0722 * b_lin

    def contrast_ratio(rgb1, rgb2):
        """Calculate WCAG contrast ratio between two RGB colors"""
        l1 = relative_luminance(*rgb1)
        l2 = relative_luminance(*rgb2)
        
        # Brighter/darker determination
        if l1 > l2:
            brighter, darker = l1, l2
        else:
            brighter, darker = l2, l1
        
        return (brighter + 0.05) / (darker + 0.05)

    # 1. Convert bg to HSL
    bg_h, bg_s, bg_l = rgb_to_hsl(*bg_rgb)
    bg_lum = relative_luminance(*bg_rgb)  # Implement as before
    
    # 2. Generate candidates
    candidates = []
    for offset in [180, 120, 240]:  # Complementary + triadic
        h = (bg_h + offset) % 360
        s = max(0.7, bg_s * 0.5 + 0.5)  # Boost saturation
        l = 0.85 if bg_l < 0.45 else 0.15  # Extreme lightness
        candidates.append((h, s, l))
    
    # 3. Adjust for contrast
    best = None
    best_cr = 0
    
    for (h, s, l) in candidates:
        # Binary search for optimal lightness
        low, high = 0.0, 1.0
        optimal_l = l
        
        for _ in range(8):
            test_l = (low + high) / 2
            rgb = hsl_to_rgb(h, s, test_l)
            fg_lum = relative_luminance(*rgb)
            cr = (max(bg_lum, fg_lum) + 0.05) / (min(bg_lum, fg_lum) + 0.05)
            
            if cr >= 4.5:
                optimal_l = test_l
                high = test_l  # Search for darker/lighter viable option
            else:
                low = test_l
        
        # Re-calculate final contrast
        final_rgb = hsl_to_rgb(h, s, optimal_l)
        final_lum = relative_luminance(*final_rgb)
        final_cr = (max(bg_lum, final_lum) + 0.05) / (min(bg_lum, final_lum) + 0.05)
        
        if final_cr >= 4.5 and final_cr > best_cr:
            best = final_rgb
            best_cr = final_cr
    
    # 4. Fallback with vibrant near-black/white
    if best is None:
        # Try vibrant dark
        dark_rgb = hsl_to_rgb(bg_h, 0.85, 0.12)
        dark_cr = contrast_ratio(bg_rgb, dark_rgb)
        if dark_cr >= 4.5:
            return dark_rgb
        
        # Try vibrant light
        light_rgb = hsl_to_rgb(bg_h, 0.85, 0.92)
        light_cr = contrast_ratio(bg_rgb, light_rgb)
        if light_cr >= 4.5:
            return light_rgb
        
        # Ultimate fallback
        return (0, 0, 0) if contrast_ratio(bg_rgb, (0,0,0)) > contrast_ratio(bg_rgb, (255,255,255)) else (255,255,255)
    
    return best

In [ ]:
# Example usage
background = (120, 80, 200)  # Purple background

# Method 1: Complementary color
vibrant_color1 = get_vibrant_contrasting_color(background)
print(f"Complementary: {vibrant_color1}")

# Method 2: CIELAB-based
vibrant_color2 = get_cielab_contrasting_color(background)
print(f"CIELAB: {vibrant_color2}")

# Method 3: Adaptive with hue shift
vibrant_color3 = get_adaptive_contrasting_color(background, hue_shift=150)
print(f"Adaptive: {vibrant_color3}")

vibrant_color4 = vibrant_contrasting_color(background)
print(f"QWEN: {vibrant_color4}")

In [ ]:
def string2hex(s):
    r = int(s[0:2], 16)
    g = int(s[2:4], 16)
    b = int(s[4:], 16)
    return (r,g,b)

def hex2string(colortuple):
    r, g, b = colortuple
    return f"{r:02X}{g:02X}{b:02X}"

def contracts(s):
    print(f"Input: {s}")
    rgb = string2hex(s)
    vibrant_color1 = get_vibrant_contrasting_color(rgb)
    print(f"Complementary: {hex2string(vibrant_color1)}")

    # Method 2: CIELAB-based
    vibrant_color2 = get_cielab_contrasting_color(rgb)
    print(f"CIELAB: {hex2string(vibrant_color2)}")

    # Method 3: Adaptive with hue shift
    vibrant_color3 = get_adaptive_contrasting_color(rgb, hue_shift=180)
    print(f"Adaptive: {hex2string(vibrant_color3)}")

    vibrant_color4 = vibrant_contrasting_color(background)
    print(f"QWEN: {hex2string(vibrant_color4)}")

contracts("F34F29")